In [24]:
import os
class_names = ['10', '9', '8', '7','6', '5', '4', '3', '2', '1']
states = [6, 3, 5, 5, 6, 5, 4, 3, 6, 3]

length = 0
for d in class_names:
    length += len(os.listdir("data/" + d))
print(length)

100


In [25]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix


In [26]:
all_data = {}
all_labels = {}
for cname in class_names:
    file_paths = [os.path.join("data", cname, i) for i in os.listdir(os.path.join('data', cname)) if i.endswith('.wav')]
    data = [get_mfcc(file_path) for file_path in file_paths]
    all_data[cname] = data
    all_labels[cname] = [class_names.index(cname) for i in range(len(file_paths))]

C:\Users\Cong\AppData\Local\Temp\ipykernel_11848\855361948.py:13: FutureWarning: Pass y=[ 0.01250432  0.01744861  0.01377264 ... -0.00988178 -0.01277887
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(
C:\Users\Cong\AppData\Local\Temp\ipykernel_11848\855361948.py:13: FutureWarning: Pass y=[0.         0.         0.         ... 0.00444902 0.0108019  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(
C:\Users\Cong\AppData\Local\Temp\ipykernel_11848\855361948.py:13: FutureWarning: Pass y=[ 4.3502441e-04  2.6727605e-04 -1.2566439e-04 ... -1.6672265e-06
  4.9939644e-07  1.7730802e-08], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(
C:\Users\Cong\AppData\Local\Temp\ipykernel_11848\855361948.py:13: FutureWa

In [27]:
from sklearn.model_selection import train_test_split

X = {'train': {}, 'test': {}}
y = {'train': {}, 'test': {}}
for cname in class_names:
    x_train, x_test, _, y_test = train_test_split(
        all_data[cname], all_labels[cname], 
        test_size = 0.33, 
        random_state=42
    )
    X['train'][cname] = x_train
    X['test'][cname] = x_test
    y['test'][cname] = y_test

In [28]:
for cname in class_names:
    print(cname,len(X['train'][cname]), len(X['test'][cname]), len(y['test'][cname]))

10 6 4 4
9 6 4 4
8 6 4 4
7 6 4 4
6 6 4 4
5 6 4 4
4 6 4 4
3 6 4 4
2 6 4 4
1 6 4 4


In [29]:
import hmmlearn.hmm as hmm

model = {}
for idx, cname in enumerate(class_names):
    start_prob_ = np.full(states[idx], 0.0)
    start_prob_[0] = 1.0
    trans_matrix = np.full((states[idx], states[idx]), 0.0)
    p = 0.5
    np.fill_diagonal(trans_matrix, p)
    np.fill_diagonal(trans_matrix[0:, 1:], 1 - p)
    trans_matrix[-1, -1] = 1.0
    
    #trans matrix
    print(cname)
    print(trans_matrix) 

    model[cname] = hmm.GaussianHMM(
        n_components=states[idx], 
        verbose=True, 
        n_iter=300, 
        startprob_prior=start_prob_, 
        transmat_prior=trans_matrix,
        params='stmc',
        init_params='mc',
        random_state=42
    )

    model[cname].fit(X=np.vstack(X['train'][cname]), lengths=[x.shape[0] for x in X['train'][cname]])

10
[[0.5 0.5 0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  1. ]]


         1      -70162.8898             +nan
         2      -65996.3045       +4166.5854
         3      -64991.8970       +1004.4075
         4      -64660.3840        +331.5130
         5      -64523.2491        +137.1349
         6      -64437.5498         +85.6993
         7      -64383.9790         +53.5708
         8      -64328.8823         +55.0967
         9      -64307.3797         +21.5026
        10      -64304.5116          +2.8681
        11      -64298.8831          +5.6285
        12      -64265.1317         +33.7515
        13      -64236.4687         +28.6630
        14      -64176.6926         +59.7761
        15      -64094.3154         +82.3773
        16      -64067.7734         +26.5420
        17      -64064.7084          +3.0649
        18      -64062.4370          +2.2714
        19      -64061.4476          +0.9895
        20      -64061.1240          +0.3236
        21      -64061.0285          +0.0955
        22      -64061.0028          +0.0258
        23

9
[[0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.  0.  1. ]]


         1      -76204.9784             +nan
         2      -72450.1753       +3754.8031
         3      -71375.6823       +1074.4929
         4      -71136.1720        +239.5103
         5      -71055.9143         +80.2577
         6      -70998.7407         +57.1736
         7      -70957.3611         +41.3796
         8      -70919.4251         +37.9360
         9      -70910.2067          +9.2184
        10      -70908.6382          +1.5685
        11      -70907.7883          +0.8498
        12      -70907.1502          +0.6381
        13      -70905.1445          +2.0057
        14      -70896.9984          +8.1461
        15      -70876.9866         +20.0117
        16      -70861.7421         +15.2445
        17      -70847.3382         +14.4039
        18      -70817.0001         +30.3381
        19      -70783.9079         +33.0922
        20      -70780.7166          +3.1913
        21      -70779.3505          +1.3661
        22      -70778.4355          +0.9150
        23

8
[[0.5 0.5 0.  0.  0. ]
 [0.  0.5 0.5 0.  0. ]
 [0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  1. ]]


...

         6      -73510.7756         +33.3681
         7      -73480.1382         +30.6374
         8      -73448.7895         +31.3487
         9      -73428.5363         +20.2532
        10      -73388.6111         +39.9252
        11      -73303.1384         +85.4727
        12      -73269.7669         +33.3715
        13      -73238.9042         +30.8628
        14      -73228.3189         +10.5853
        15      -73228.2957          +0.0232
        16      -73228.4186          -0.1229
         1      -71021.9396             +nan
         2      -68346.2232       +2675.7164
         3      -67681.0847        +665.1385
         4      -67100.7594        +580.3252
         5      -66816.8529        +283.9065
         6      -66792.1261         +24.7268
         7      -66788.7109          +3.4152
         8      -66785.4040          +3.3068
         9      -66779.7125          +5.6915
        10      -66777.5301          +2.1824
        11      -66777.4009          +0.1292
        12

2
[[0.5 0.5 0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  1. ]]


         5      -69703.4188        +114.4993
         6      -69473.8115        +229.6073
         7      -69336.4646        +137.3469
         8      -69305.6009         +30.8637
         9      -69269.0322         +36.5686
        10      -69180.3393         +88.6930
        11      -69124.3486         +55.9906
        12      -69111.7044         +12.6442
        13      -69098.1243         +13.5801
        14      -69096.6611          +1.4632
        15      -69093.5244          +3.1367
        16      -69089.3232          +4.2013
        17      -69086.0336          +3.2896
        18      -69084.8096          +1.2239
        19      -69084.1926          +0.6170
        20      -69083.4693          +0.7233
        21      -69082.2277          +1.2416
        22      -69075.8741          +6.3536
        23      -69073.3055          +2.5686
        24      -69071.1917          +2.1137
        25      -69069.2603          +1.9315
        26      -69064.6301          +4.6302
        27

1
[[0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.  0.  1. ]]


         1      -66576.9210             +nan
         2      -62733.8631       +3843.0579
         3      -62160.7206        +573.1424
         4      -62012.4959        +148.2248
         5      -61969.4340         +43.0619
         6      -61947.0266         +22.4074
         7      -61923.4125         +23.6141
         8      -61896.5353         +26.8773
         9      -61883.4975         +13.0378
        10      -61879.6933          +3.8042
        11      -61879.1192          +0.5741
        12      -61879.0440          +0.0752
        13      -61879.0312          +0.0128
        14      -61879.0288          +0.0024


In [30]:
import pickle

# save model
for cname in class_names:
    name = f'models_train\model_{cname}.pkl'
    name = f'models\model_{cname}.pkl'
    with open(name, 'wb') as file: 
        pickle.dump(model[cname], file)

In [35]:
# import csv
# import json
# for cname in class_names:
#     name = f'models_train\model_{cname}.csv'
#     name = f'models\model_{cname}.csv'
#     with open(name, 'w') as f:
#         f.write(json.dumps(model[cname]))

TypeError: Object of type GaussianHMM is not JSON serializable

In [ ]:
import pickle, os
import numpy as np

from sklearn.metrics import classification_report

In [ ]:
y_true = []
y_pred = []
for cname in class_names:
    for mfcc, target in zip(X['test'][cname], y['test'][cname]):
        scores = [model[cname].score(mfcc) for cname in class_names]
        pred = np.argmax(scores)
        y_pred.append(pred)
        y_true.append(target)
    print((np.array(y_true) == np.array(y_pred)).sum()/len(y_true))
print(y_true)
print(y_pred)


0.5
0.625
0.4166666666666667
0.375
0.45
0.4583333333333333
0.4642857142857143
0.46875
0.5277777777777778
0.525
[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9]
[6, 0, 8, 0, 6, 1, 1, 1, 5, 5, 8, 7, 8, 0, 8, 3, 4, 4, 8, 4, 5, 0, 7, 5, 9, 6, 9, 6, 7, 0, 7, 5, 8, 8, 8, 8, 0, 6, 9, 9]


In [ ]:
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

          10       0.33      0.50      0.40         4
           9       1.00      0.75      0.86         4
           8       0.00      0.00      0.00         4
           7       1.00      0.25      0.40         4
           6       1.00      0.75      0.86         4
           5       0.40      0.50      0.44         4
           4       0.40      0.50      0.44         4
           3       0.50      0.50      0.50         4
           2       0.44      1.00      0.62         4
           1       0.50      0.50      0.50         4

    accuracy                           0.53        40
   macro avg       0.56      0.53      0.50        40
weighted avg       0.56      0.53      0.50        40



c:\Users\Cong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Cong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Cong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
#loadmodels
import pickle

model_train = {}
for key in class_names:
    name = f"models\model_{key}.pkl"
    with open(name, 'rb') as file:
        model_train[key] = pickle.load(file)

In [ ]:
print(model_train)

{'10': GaussianHMM(init_params='mc', n_components=6, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0., 0., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 1. ]]),
            verbose=True), '9': GaussianHMM(init_params='mc', n_components=3, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. ],
       [0. , 0.5, 0.5],
       [0. , 0. , 1. ]]),
            verbose=True), '8': GaussianHMM(init_params='mc', n_components=5, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0.5, 0.5],
       [0.

In [ ]:
# import pickle as pkl
# import pandas as pd
# with open("model_cothe.pkl", "rb") as f:
#     object = pkl.load(f)
    
# df = pd.DataFrame(object)
# df.to_csv(r'models\model_cothe.csv')